In [1]:
import cv2
import numpy as np
import tensorflow as tf
import json


/Users/vineetkumar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
model = tf.keras.models.load_model("waste_model_merged.keras")

with open("label_map_merged.json") as f:
    label_map = json.load(f)

label_map = {int(k): v for k, v in label_map.items()}
IMG_SIZE = 224

print("Loaded classes:", label_map)


Loaded classes: {0: 'Aerosol', 1: 'Aluminium blister pack', 2: 'Aluminium foil', 3: 'Battery', 4: 'Broken glass', 5: 'Carded blister pack', 6: 'Cigarette', 7: 'Clear plastic bottle', 8: 'Corrugated carton', 9: 'Crisp packet', 10: 'Disposable food container', 11: 'Disposable plastic cup', 12: 'Drink can', 13: 'Drink carton', 14: 'Egg carton', 15: 'Foam cup', 16: 'Foam food container', 17: 'Food Can', 18: 'Food waste', 19: 'Garbage bag', 20: 'Glass bottle', 21: 'Glass cup', 22: 'Glass jar', 23: 'Magazine paper', 24: 'Meal carton', 25: 'Metal bottle cap', 26: 'Metal lid', 27: 'Normal paper', 28: 'Other carton', 29: 'Other plastic', 30: 'Other plastic bottle', 31: 'Other plastic container', 32: 'Other plastic cup', 33: 'Other plastic wrapper', 34: 'Paper bag', 35: 'Paper cup', 36: 'Paper straw', 37: 'Pizza box', 38: 'Plastic bottle cap', 39: 'Plastic film', 40: 'Plastic glooves', 41: 'Plastic lid', 42: 'Plastic straw', 43: 'Plastic utensils', 44: 'Plastified paper bag', 45: 'Polypropylene 

In [3]:
def predict_frame(frame):
    img = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img, verbose=0)[0]
    class_id = int(np.argmax(pred))
    return label_map[class_id]


In [4]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    label = predict_frame(frame)

    cv2.putText(frame, label, (10,40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1, (0,255,0), 2)

    cv2.imshow("Waste Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
